To run this Fenic demo, click **Runtime** > **Run all**.

<div class="align-center">
<a href="https://github.com/typedef-ai/fenic"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/typedef-fenic-logo-github-yellow.png" height="50"></a>
<a href="https://discord.gg/GdqF3J7huR"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/join-the-discord.png" height="50"></a>
<a href="https://docs.fenic.ai/latest/"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/documentation.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/typedef-ai/fenic).

</div>

In [ ]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn google-genai
!pip install polars==1.30.0
# === GOOGLE GEMINI ===
#!pip install fenic[google]
# === ANTHROPIC CLAUDE ===
#!pip install fenic[anthropic]
# === OPENAI (Default) ===
!pip install fenic

In [ ]:
import os 
import getpass

# 🔌 MULTI-PROVIDER SETUP - Choose your preferred LLM provider
# Uncomment ONE of the provider sections below:

# === OPENAI (Default) ===
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# === GOOGLE GEMINI ===
# os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")

# === ANTHROPIC CLAUDE ===
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")

# 📧 Email Thread Categorization

**Hook:** *"Turn your overflowing inbox into organized, actionable intelligence"*

Email chaos is real - customer complaints mixed with feature requests, urgent issues buried in newsletters, action items scattered across long threads. Traditional email filters use simple rules. Watch AI understand context, intent, and urgency to automatically organize your communications.

**What you'll see in this 2-minute demo:**
- 📬 **Mixed email threads** - Customer support, internal discussions, vendor communications
- 🎯 **Smart categorization** - Beyond subject lines, understanding content and intent
- ⚡ **Urgency detection** - Automatic priority assessment
- 📊 **Thread insights** - Sentiment, action items, and follow-up requirements

Perfect for customer support, executive assistance, and email-heavy workflows.

In [ ]:
import fenic as fc
from pydantic import BaseModel, Field
from typing import List, Literal

# ⚡ Configure for email analysis
session = fc.Session.get_or_create(fc.SessionConfig(
    app_name="email_categorization_demo",
    semantic=fc.SemanticConfig(
        language_models={
            "email_ai": fc.OpenAILanguageModel(model_name="gpt-4o-mini", rpm=500, tpm=200_000),
            # "email_ai": fc.GoogleDeveloperLanguageModel(model_name="gemini-2.5-flash-lite", rpm=1000, tpm=1_000_000),
            # "email_ai": fc.AnthropicLanguageModel(model_name="claude-3-5-sonnet-20241022", rpm=500, tpm=200_000)
        }
    )
))

print("✅ Email thread categorization session configured")

## 📋 Step 1: Email Analysis Schema

Define the structured insights we want to extract from email threads:

In [ ]:
# 📧 Email thread analysis schema
class EmailThreadAnalysis(BaseModel):
    thread_category: Literal["customer_support", "feature_request", "bug_report", "internal_discussion", "vendor_communication", "newsletter"] = Field(description="Primary thread category")
    urgency_level: Literal["critical", "high", "medium", "low"] = Field(description="Response urgency required")
    requires_action: bool = Field(description="Whether thread requires action or response")
    sentiment: Literal["positive", "negative", "neutral", "frustrated", "appreciative"] = Field(description="Overall emotional tone")
    key_topics: List[str] = Field(description="Main topics or issues discussed")
    mentioned_people: List[str] = Field(description="People mentioned in the thread")
    business_impact: Literal["revenue_affecting", "user_experience", "operational", "informational"] = Field(description="Business impact level")
    suggested_response_time: Literal["immediate", "same_day", "within_week", "no_response_needed"] = Field(description="Recommended response timeframe")
    thread_summary: str = Field(description="One-sentence summary of the thread")

print("📧 Email Thread Analysis Schema:")
print("   • thread_category: Primary classification of email content")
print("   • urgency_level: Response priority assessment")
print("   • requires_action: Action flag for workflow routing")
print("   • sentiment: Emotional tone analysis")
print("   • key_topics: Main discussion points")
print("   • mentioned_people: Stakeholder identification")
print("   • business_impact: Impact on business operations")
print("   • suggested_response_time: Response SLA guidance")
print("   • thread_summary: Executive summary")

## 📬 Step 2: Mixed Email Thread Collection

Realistic email threads spanning different categories and urgency levels:

In [ ]:
# 📬 Email threads with varying categories and priorities
email_threads = session.create_dataframe([
    {
        "thread_id": "EMAIL001",
        "subject": "URGENT: Production API Down",
        "participants": "ops@company.com, cto@company.com, support@company.com",
        "thread_content": """From: ops@company.com
Subject: URGENT: Production API Down

Our production API started returning 500 errors at 2:47 PM PST. Multiple customers are affected. Initial investigation shows database connection timeouts. All hands needed.

From: cto@company.com
I'm investigating. Database logs show connection pool exhaustion. Scaling up DB instances now.

From: support@company.com
We have 12 customer tickets opened in the last 30 minutes. Enterprise clients TechCorp and DataFlow are particularly concerned. Need ETA for resolution.

From: ops@company.com
Database scaling complete. API response times back to normal. Monitoring for next 2 hours. Root cause: traffic spike from new customer integration."""
    },
    {
        "thread_id": "EMAIL002", 
        "subject": "Feature Request: Dark Mode Support",
        "participants": "customer@techflow.com, support@company.com, product@company.com",
        "thread_content": """From: customer@techflow.com
Subject: Feature Request: Dark Mode Support

Hi team, our developers spend long hours using your dashboard and would love dark mode support. This would really improve the user experience, especially during late-night debugging sessions. Is this something on your roadmap?

From: support@company.com
Thanks for the feedback! I've forwarded this to our product team. Dark mode has come up in other customer conversations too.

From: product@company.com
We appreciate the suggestion! Dark mode is actually planned for our Q2 release. I'll add TechFlow to our beta testing list when it's ready. Expected timeline is April-May 2024.

From: customer@techflow.com
That's fantastic! We'd love to participate in beta testing. Our team is very excited about this feature."""
    },
    {
        "thread_id": "EMAIL003",
        "subject": "Q1 Marketing Budget Discussion",
        "participants": "marketing@company.com, finance@company.com, ceo@company.com", 
        "thread_content": """From: marketing@company.com
Subject: Q1 Marketing Budget Discussion

Team, we need to finalize Q1 marketing spend allocation. Proposal: 40% digital advertising, 30% content creation, 20% events, 10% PR. Total budget $50K.

From: finance@company.com
Budget approved, but let's track ROI closely. Last quarter's event spend didn't convert well. Suggest shifting 5% from events to digital.

From: ceo@company.com
I agree with finance. Also, prioritize content that showcases our new AI features. The market is very receptive to AI messaging right now.

From: marketing@company.com
Perfect. Revised allocation: 45% digital, 30% content (AI-focused), 15% events, 10% PR. I'll prepare the detailed plan by Friday and share for final approval."""
    },
    {
        "thread_id": "EMAIL004",
        "subject": "Billing Issue - Account Suspended",
        "participants": "angry.customer@startup.com, billing@company.com, support@company.com",
        "thread_content": """From: angry.customer@startup.com
Subject: Billing Issue - Account Suspended

This is completely unacceptable! My account was suspended for non-payment, but I have proof my credit card was charged on January 15th. Your billing system is clearly broken and this is affecting my business operations. I demand immediate resolution!

From: billing@company.com
I sincerely apologize for this issue. I've located your payment - there was a processing delay that caused the suspension. Your account has been immediately restored and I'm adding a $50 credit for the inconvenience.

From: support@company.com
I've also checked your service logs - no data was lost during the suspension period. Everything is back to normal. Again, very sorry for this disruption.

From: angry.customer@startup.com
Thank you for the quick resolution and the credit. I appreciate you taking this seriously. The service is back up and running perfectly."""
    },
    {
        "thread_id": "EMAIL005",
        "subject": "Monthly Newsletter - Product Updates",
        "participants": "newsletter@company.com, subscribers@list.com",
        "thread_content": """From: newsletter@company.com
Subject: Monthly Newsletter - Product Updates

📰 January 2024 Product Updates

🚀 New Features:
- Advanced analytics dashboard with custom metrics
- Improved API response times (30% faster)
- Enhanced mobile app with offline support

📊 Customer Spotlight:
DataFlow Solutions increased efficiency by 40% using our automation features. Read their case study on our blog.

📅 Upcoming:
- Webinar: "AI-Powered Data Analysis" on February 15th
- Beta testing for dark mode feature
- New integration with popular CRM platforms

Questions? Reply to this email or visit our support portal.

Best regards,
The Product Team"""
    }
])

print("📬 Email Thread Collection - Mixed categories and priorities:")
email_threads.select("thread_id", "subject", "participants").show()

## 🧠 Step 3: Intelligent Email Categorization

Analyze email threads to extract category, urgency, and business context:

In [ ]:
# 🧠 Extract insights from email threads
email_analysis = email_threads.select(
    "thread_id",
    "subject", 
    "participants",
    fc.semantic.extract(
        "thread_content",
        EmailThreadAnalysis,
        model_alias="email_ai"
    ).alias("analysis")
).cache()

# Show categorization and priority assessment
email_overview = email_analysis.select(
    "thread_id",
    "subject",
    email_analysis.analysis.thread_category.alias("category"),
    email_analysis.analysis.urgency_level.alias("urgency"),
    email_analysis.analysis.business_impact.alias("impact"),
    email_analysis.analysis.suggested_response_time.alias("response_time")
)

print("🧠 EMAIL CATEGORIZATION AND PRIORITY:")
email_overview.show()

## ⚡ Step 4: Actionable Email Intelligence

Identify emails requiring immediate attention and organize by business impact:

In [ ]:
# ⚡ Analyze email characteristics for action planning
email_insights = email_analysis.select(
    "thread_id",
    "subject",
    email_analysis.analysis.requires_action.alias("action_needed"),
    email_analysis.analysis.sentiment.alias("sentiment"),
    email_analysis.analysis.key_topics.alias("topics"),
    email_analysis.analysis.thread_summary.alias("summary")
)

print("⚡ EMAIL ACTION INTELLIGENCE:")
email_insights.show()

# Email workload analysis
total_emails = email_threads.count()
action_required = email_analysis.filter(email_analysis.analysis.requires_action).count()
critical_urgent = email_analysis.filter(
    email_analysis.analysis.urgency_level.is_in(["critical", "high"])
).count()
revenue_affecting = email_analysis.filter(
    email_analysis.analysis.business_impact == "revenue_affecting"
).count()

print("\n📊 EMAIL WORKLOAD ANALYSIS:")
print(f"   • Total email threads: {total_emails}")
print(f"   • Requiring action/response: {action_required} ({action_required/total_emails*100:.0f}%)")
print(f"   • Critical/high urgency: {critical_urgent} ({critical_urgent/total_emails*100:.0f}%)")
print(f"   • Revenue-affecting: {revenue_affecting}")

print("\n🎯 BUSINESS BENEFITS:")
print("   • Automatic email triage by urgency and category")
print("   • Business impact assessment for prioritization")
print("   • Sentiment analysis identifies customer satisfaction issues")
print("   • Response time suggestions optimize customer service")
print("   • Action flags prevent important emails from being missed")
print("   • Topic extraction enables better team routing")
print("   • People mentions facilitate proper escalation")

In [ ]:
session.stop()